In [4]:
from faker import Faker
import csv

faker = Faker()

# Configure how many Trinity Bank customers there are making / receiving transactions
numberTrinityBankAC = 500
#Trinity a/cs 8 digit with 2x leading zeros

# Specify the range for the random number of transactions generated with external accounts
minNumTx = 30
maxNumTx = 100
#External a/cs are 9 digit with 1x leading zero

# Specify the max number of transactions generated between Trinity accounts (the min. is 1)
maxTrinityTx = 10
# proportion of Trinity accounts that will transact with each other
propTrinityAc = 0.2

currency = "USD"  

#output files
transactionFile = 'output/transactions.csv'
customerFile = 'output/customers.csv'

def createTransactions(accountNum):   
    #The Trinity customer account we want to generate some transactions for
    fromAccount=accountNum
    
    # Create a random number (in the range of) of outgoing transactions to external bank a/c 
    numOutgoingTXs = faker.random_int(minNumTx,maxNumTx,1)
    # Create a random number of incoming transactions from external bank a/c
    numIncomingTXs = faker.random_int(minNumTx,maxNumTx,1)
    # These transactions will be spread over 3 months
    
    #open a file and append generated transactions for this customer (csv format)   
    with open(transactionFile, mode='a', newline='') as tx_file:
        tx_writer = csv.writer(tx_file, delimiter=',') 
        print("Generating", numOutgoingTXs,"outgoing transactions FROM Trinity Bank Customer: ",fromAccount)
        for _ in range(numOutgoingTXs):
            amount=faker.pricetag()
            #remove the . and $ from the random pricetag
            amount=amount.replace(",","")
            amount=amount.replace("$","")
            #Our Trinity customer is paying a random external account with a 9 digit a/c no. with leading zero
            toAccount = faker.numerify('0%########') 
            transactionID = faker.numerify('%###########')
            #create transactions for a period of 3 months so we can shard all transactions by month
            dateTime=faker.date_time_between(start_date='-90d', end_date='now')
            #print(fromAccount,amount,transactionID,toAccount,dateTime)
            tx_writer.writerow([fromAccount,amount,currency,transactionID,toAccount,dateTime])
        print("Generating", numIncomingTXs,"incoming TO transactions for Trinity Bank Customer: ",accountNum)
        for _ in range(numIncomingTXs):
            amount=faker.pricetag()
            amount=amount.replace(",","")
            amount=amount.replace("$","")
            #This time we want the toAccount to be our Trinity Bank customer i.e the account we pass in
            toAccount = accountNum
            #Our Trinity customer is being paid by a random external account with a 9 digit a/c no. with leading zero
            fromAccount = faker.numerify('0%#######')
            transactionID = faker.numerify('%###########')
            #create transactions for a period of 3 months so we can shard all transactions by month
            dateTime=faker.date_time_between(start_date='-90d', end_date='now')
            #print(fromAccount,amount,transactionID,toAccount,dateTime)
            tx_writer.writerow([fromAccount,amount,currency,transactionID,toAccount,dateTime])

def createTransactionsAtRandomBetweenTrinityCustomers(fromAccount,toAccount):
    #random number of tx between 1 and maxTrinityTx 
    numTXs = faker.random_int(1,maxTrinityTx,1)
    # append to the output transaction file
    with open(transactionFile, mode='a', newline='') as tx_file:
        tx_writer = csv.writer(tx_file, delimiter=',')
        print("Generating", numTXs,"BETWEEN Trinity Bank Customer: ",fromAccount,"and",toAccount)
        for _ in range(numTXs):
            amount=faker.pricetag()
            #remove the , and $
            amount=amount.replace(",","")
            amount=amount.replace("$","")
            transactionID = faker.numerify('%###########')
            #TO DO create them on proper month boundaries
            #create transactions for a period of 3 months so we can shard all transactions by month
            dateTime=faker.date_time_between(start_date='-90d', end_date='now')
            #print(fromAccount,amount,transactionID,toAccount,dateTime)
            tx_writer.writerow([fromAccount,amount,currency,transactionID,toAccount,dateTime])

        
def createTrinityBankCustomers(numAccounts):
    #maintain a list of customer a/c nos.
    listTrinityCustomers = []
    #overwrite any pre-existing customer file
    with open(customerFile, mode='w', newline='') as cust_file:
        cust_writer = csv.writer(cust_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        cust_writer.writerow(['name','customerID','address','email','phoneNumber','account','sortCode'])

        # generate customer accounts
        for _ in range(numAccounts):
            name = faker.name()
            customerID = faker.uuid4()
            address = faker.address().replace(',', ' ')
            address = faker.address().replace('\n', ' ')
            email = faker.ascii_email()
            phoneNumber = faker.phone_number()
            #Trinity a/cs 8 digit with 2x leading zeros
            account = faker.numerify('00%%###%')
            listTrinityCustomers.append(account)
            #going to assign a sort code on the account but for simplicity we wont use it in transactions
            #only last two digits random as the customers for this bank all have similar sort codes
            sortCode = faker.numerify('80-10-##')
            #print(name,customerID,address,account,sortCode)
            print("Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)")
            cust_writer.writerow([name,customerID,address,email,phoneNumber,account,sortCode])
            createTransactions(account)
    #generate transactions between 1/5 of Trinity Bank customers
    numInterBankTXs=int(numberTrinityBankAC/5)
    print("-----\n")
    print("Generate txs BETWEEN",numInterBankTXs,"Trinity customers")
    for _ in range(numInterBankTXs):
        fromTrinityAccount=faker.random_element(elements=listTrinityCustomers)
        toTrinityAccount=faker.random_element(elements=listTrinityCustomers)
        createTransactionsAtRandomBetweenTrinityCustomers(fromTrinityAccount,toTrinityAccount)

# Write out the config        
print("Creating", numberTrinityBankAC,"Trinity Bank a/c")
print("Using",currency,"for transactions")
print("Min number of ",minNumTx,"transactions with external a/cs")
print("Max number of ",maxNumTx,"transactions with external a/cs")
print("Max number of ",maxTrinityTx,"transactions between Trinity a/cs")
print("Proportion of Trinity a/c that transact with each other ",propTrinityAc)
print("-----\n")
       
#create the header in the transaction file - and overwrite any previous records 
print("Creating transaction file header (overwritting previous file)")
with open(transactionFile, mode='w+', newline='') as tx_file:
    tx_writer = csv.writer(tx_file, delimiter=',') #, quotechar='"', quoting=csv.QUOTE_MINIMAL)
    tx_writer.writerow(['fromAccount','amount','currency','transactionID','toAccount','dateTime'])   
        
createTrinityBankCustomers(numberTrinityBankAC)


Creating 500 Trinity Bank a/c
Using USD for transactions
Min number of  30 transactions with external a/cs
Max number of  100 transactions with external a/cs
Max number of  10 transactions between Trinity a/cs
Proportion of Trinity a/c that transact with each other  0.2
-----

Creating transaction file header (overwritting previous file)
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 82 outgoing transactions FROM Trinity Bank Customer:  00290699
Generating 63 incoming TO transactions for Trinity Bank Customer:  00290699
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 100 outgoing transactions FROM Trinity Bank Customer:  00839676
Generating 40 incoming TO transactions for Trinity Bank Customer:  00839676
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 37 outgoing transactions FROM Trinity Bank Customer:  00781742
Generating 40 incoming TO transactions for Trinity Ban

Generating 99 incoming TO transactions for Trinity Bank Customer:  00516114
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 96 outgoing transactions FROM Trinity Bank Customer:  00579459
Generating 91 incoming TO transactions for Trinity Bank Customer:  00579459
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 56 outgoing transactions FROM Trinity Bank Customer:  00319513
Generating 50 incoming TO transactions for Trinity Bank Customer:  00319513
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 48 outgoing transactions FROM Trinity Bank Customer:  00515393
Generating 89 incoming TO transactions for Trinity Bank Customer:  00515393
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 31 outgoing transactions FROM Trinity Bank Customer:  00772142
Generating 53 incoming TO transactions for Trinity Bank Customer:  00772142
Generating custo

Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 58 outgoing transactions FROM Trinity Bank Customer:  00567301
Generating 90 incoming TO transactions for Trinity Bank Customer:  00567301
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 82 outgoing transactions FROM Trinity Bank Customer:  00961495
Generating 62 incoming TO transactions for Trinity Bank Customer:  00961495
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 62 outgoing transactions FROM Trinity Bank Customer:  00677651
Generating 31 incoming TO transactions for Trinity Bank Customer:  00677651
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 33 outgoing transactions FROM Trinity Bank Customer:  00860264
Generating 46 incoming TO transactions for Trinity Bank Customer:  00860264
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 32 o

Generating 71 incoming TO transactions for Trinity Bank Customer:  00846011
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 42 outgoing transactions FROM Trinity Bank Customer:  00172983
Generating 58 incoming TO transactions for Trinity Bank Customer:  00172983
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 51 outgoing transactions FROM Trinity Bank Customer:  00234778
Generating 44 incoming TO transactions for Trinity Bank Customer:  00234778
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 91 outgoing transactions FROM Trinity Bank Customer:  00756704
Generating 80 incoming TO transactions for Trinity Bank Customer:  00756704
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 58 outgoing transactions FROM Trinity Bank Customer:  00542419
Generating 46 incoming TO transactions for Trinity Bank Customer:  00542419
Generating custo

Generating 71 incoming TO transactions for Trinity Bank Customer:  00897975
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 52 outgoing transactions FROM Trinity Bank Customer:  00813887
Generating 100 incoming TO transactions for Trinity Bank Customer:  00813887
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 56 outgoing transactions FROM Trinity Bank Customer:  00893252
Generating 97 incoming TO transactions for Trinity Bank Customer:  00893252
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 54 outgoing transactions FROM Trinity Bank Customer:  00965276
Generating 86 incoming TO transactions for Trinity Bank Customer:  00965276
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 87 outgoing transactions FROM Trinity Bank Customer:  00713607
Generating 55 incoming TO transactions for Trinity Bank Customer:  00713607
Generating cust

...

Generating 33 incoming TO transactions for Trinity Bank Customer:  00559969
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 30 outgoing transactions FROM Trinity Bank Customer:  00941822
Generating 95 incoming TO transactions for Trinity Bank Customer:  00941822
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 56 outgoing transactions FROM Trinity Bank Customer:  00285991
Generating 46 incoming TO transactions for Trinity Bank Customer:  00285991
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 45 outgoing transactions FROM Trinity Bank Customer:  00258684
Generating 38 incoming TO transactions for Trinity Bank Customer:  00258684
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 95 outgoing transactions FROM Trinity Bank Customer:  00540133
Generating 81 incoming TO transactions for Trinity Bank Customer:  00540133
Generating custo

Generating 97 incoming TO transactions for Trinity Bank Customer:  00881556
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 31 outgoing transactions FROM Trinity Bank Customer:  00166631
Generating 69 incoming TO transactions for Trinity Bank Customer:  00166631
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 91 outgoing transactions FROM Trinity Bank Customer:  00697215
Generating 33 incoming TO transactions for Trinity Bank Customer:  00697215
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 67 outgoing transactions FROM Trinity Bank Customer:  00827248
Generating 69 incoming TO transactions for Trinity Bank Customer:  00827248
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 74 outgoing transactions FROM Trinity Bank Customer:  00479531
Generating 95 incoming TO transactions for Trinity Bank Customer:  00479531
Generating custo

Generating 56 incoming TO transactions for Trinity Bank Customer:  00763842
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 59 outgoing transactions FROM Trinity Bank Customer:  00121781
Generating 88 incoming TO transactions for Trinity Bank Customer:  00121781
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 30 outgoing transactions FROM Trinity Bank Customer:  00357982
Generating 74 incoming TO transactions for Trinity Bank Customer:  00357982
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 75 outgoing transactions FROM Trinity Bank Customer:  00654286
Generating 91 incoming TO transactions for Trinity Bank Customer:  00654286
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 69 outgoing transactions FROM Trinity Bank Customer:  00573186
Generating 71 incoming TO transactions for Trinity Bank Customer:  00573186
Generating custo

Generating 72 incoming TO transactions for Trinity Bank Customer:  00962607
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 34 outgoing transactions FROM Trinity Bank Customer:  00296176
Generating 47 incoming TO transactions for Trinity Bank Customer:  00296176
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 77 outgoing transactions FROM Trinity Bank Customer:  00814259
Generating 52 incoming TO transactions for Trinity Bank Customer:  00814259
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 65 outgoing transactions FROM Trinity Bank Customer:  00787085
Generating 64 incoming TO transactions for Trinity Bank Customer:  00787085
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 50 outgoing transactions FROM Trinity Bank Customer:  00564716
Generating 71 incoming TO transactions for Trinity Bank Customer:  00564716
Generating custo

Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 61 outgoing transactions FROM Trinity Bank Customer:  00298535
Generating 55 incoming TO transactions for Trinity Bank Customer:  00298535
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 78 outgoing transactions FROM Trinity Bank Customer:  00983836
Generating 39 incoming TO transactions for Trinity Bank Customer:  00983836
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 75 outgoing transactions FROM Trinity Bank Customer:  00245229
Generating 67 incoming TO transactions for Trinity Bank Customer:  00245229
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 88 outgoing transactions FROM Trinity Bank Customer:  00858591
Generating 42 incoming TO transactions for Trinity Bank Customer:  00858591
Generating customer for Trinity Bank (8 digit a/c no with two leading zeros)
Generating 94 o